# 1 - Preparando ambiente

## Instalar requisitos

In [ ]:
%pip install -r requirements.txt

## Importar biblioteca

In [ ]:
from main import eContabilSite, insistir

## Definir senha

Para que o processo possa ser executado corretamente, é preciso ter um arquivo config.json com o domínio, usuário e senha do ambiente informados para que a plataforma possa ser acessada.

Use o arquivo ```config_sample.json```para se basear ao criar o arquivo ```config.json``` corretamente.

# 2 - Iniciando a exportação

In [ ]:
print('Define o Log de erros como falso. (exibe erros e status no console)')
dbg = False
print('Carrega a classe, abre o navegador, loga na plataforma.')
es = eContabilSite(dbg)

print('Cria tabelas de acompanhamento do processo no banco de dados')
es.create_tables()

In [ ]:
print('Raspar dados de Clientes')
es.get_clients()

print('Raspar dados dos eventos')
es.get_movfolha()

print('Raspar dados dos outros pagamentos')
es.get_outros_pag()

print('Raspar dados dos outros pagamentos')
es.get_outros_pag()

print('Raspar dados dos usuários')
es.get_user()

## Obter movimento

Esta etapa é a mais demorada. Ela é responsável por baixar os arquivos

**IMPORTANTE:** requer que a função ```.get_clients()``` já tenha sido chamada antes.

O Exemlo abaixo usa o método ```insistir``` da biblioteca main. Este método foi criado para invocar várias vezes a função ```.get_mov()``` da classe eContabilSite, visto que o site parece parar de responder em alguns casos. Devido a ocorrência infrquente de erros e baixa reutilização deste código, optou-se por não tratar o erro e usar a força bruta (chamar novamente a função, até dar certo). Usando este método, pode-se obter em média 490 Protocolos e 490 recibos para cada hora de execução do código (imprimindo como PDF).

In [ ]:
# Caso queira pode dividir a carga em mais de um executor pyhton, nos meus testes não fez diferença, portanto pode usar 1
quant_to_split = 1 
number_bot = 0

anos = list(range(2014,2020))  #Obter dados de 2014 à 2019
meses = list(range(1,14)) #Obter dados do mês 1 ao mês 13

es = insistir(quant_to_split=quant_to_split, number_bot=number_bot, anos=anos, meses=meses)